In [ ]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, ToolMessage

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client via LangChain
model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    temperature=0
)

### Learn

In [44]:
model.invoke("The sky is").content

'blue and vast, often filled with fluffy white clouds during the day, and dotted with stars at night.'

In [45]:
from langchain_core.messages import HumanMessage, SystemMessage

system_msg = SystemMessage(
    '''You are a helpful assistant that responds to questions with three 
        exclamation marks.'''
)
human_msg = HumanMessage('What is the capital of France?')

model.invoke([system_msg, human_msg]).content

'The capital of France is Paris!!!'

In [46]:
from langchain_core.prompts import ChatPromptTemplate

# both `template` and `model` can be reused many times

template = ChatPromptTemplate.from_messages([
    ('system', '''Answer the question based on the context below. If the 		
        question cannot be answered using the information provided, answer
        with "I don\'t know".'''),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])

# `prompt` and `completion` are the results of using template and model once

prompt = template.invoke({
    "context": """The most recent advancements in NLP are being driven by 
        Large Language Models (LLMs). These models outperform their smaller 
        counterparts and have become invaluable for developers who are creating 
        applications with NLP capabilities. Developers can tap into these 
        models through Hugging Face's `transformers` library, or by utilizing 
        OpenAI and Cohere's offerings through the `openai` and `cohere` 
        libraries, respectively.""",
    "question": "Which model providers offer LLMs?"
})

model.invoke(prompt).content

'Hugging Face, OpenAI, and Cohere offer LLMs.'

### Runnable Interface

In [57]:
completion = model.invoke('Hi there!').content
# Hi!

print(completion)

Hello! How can I assist you today?


In [60]:
completions = model.batch(['Hi there!', 'Bye!'])
# ['Hi!', 'See you!']

for message in completions:
    print(message.content)

Hello! How can I assist you today?
Goodbye! If you have any more questions in the future, feel free to ask. Have a great day!


In [61]:
for token in model.stream('Bye!'):
    print(token.content, end="")
    # Good
    # bye
    # !

Goodbye! If you have any more questions in the future, feel free to ask. Have a great day!

# TLOF

In [47]:
from langchain_core.prompts import ChatPromptTemplate

# both `template` and `model` can be reused many times

template = ChatPromptTemplate.from_messages([
    ('system', '''You are a civil aviation layout assistant that helps engineers, architects, and designers generate 3D-ready TLOF (Touchdown and Lift-Off Area) configurations from plain English descriptions. 
                You understand and convert natural language into a structured JSON format for heliport design, including parameters like geometry, markings, lighting, safety areas, and landing markers. 
                You respect strict validation rules for shape types, colors, marker types, dimensions, and category activations.
                Use default values where input is missing or unclear, and validate all parameters with explanations. All outputs should follow the `generate_landing_surface_layout` function schema.
            '''),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])

# `prompt` and `completion` are the results of using template and model once

prompt = template.invoke({
    "context": """Describe the TLOF landing surface:
                I need a polygonal landing surface for a large helicopter. Make it a hexagon with a 30-meter diameter, located at [100.75, 13.76]. Set the elevation to 1.2 meters and make it fully opaque. Use dashed yellow markings, and add a landing marker with a red “V”, rotated 90 degrees. Include basic lighting and a TDPC in green shaped as a cross. Add a safety area using a multiplier of 1.2 with a d-value of 12.
                """,
    "question": "Based on the user’s description, generate a structured JSON layout for the TLOF, applying the correct schema, value ranges, and defaults. Use the `generate_landing_surface_layout` function definition to guide parameter formatting and validation."
})

print(model.invoke(prompt).content)

To generate the structured JSON layout for the TLOF based on the provided description, we need to validate and structure each component of the request:

1. **Geometry**: The landing surface is described as a hexagon with a 30-meter diameter. The hexagon shape is valid, and the diameter is within a reasonable range for a large helicopter.

2. **Location and Elevation**: The coordinates [100.75, 13.76] are provided with an elevation of 1.2 meters. These values are valid.

3. **Opacity**: The surface is described as fully opaque, which is valid.

4. **Markings**: Dashed yellow markings are specified. Both the style ("dashed") and color ("yellow") are valid.

5. **Landing Marker**: A red “V” marker, rotated 90 degrees, is requested. The color "red" and rotation are valid. The marker type "V" is a standard type.

6. **Lighting**: Basic lighting is mentioned without specifics, so default lighting settings will be used.

7. **TDPC (Touchdown Positioning Circle)**: A green cross is specified. 

### Truly LLM based JSON output

In [48]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Corrected prompt with properly escaped JSON
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a civil aviation layout assistant that helps engineers, architects, and designers generate 3D-ready TLOF (Touchdown and Lift-Off Area) configurations from plain English descriptions and also that generates TLOF configurations in JSON format. 
                You understand and convert natural language into a structured JSON format for heliport design, including parameters like geometry, markings, lighting, safety areas, and landing markers. 
                You respect strict validation rules for shape types, colors, marker types, dimensions, and category activations.
                Use default values where input is missing or unclear, and validate all parameters with explanations.

Example Input:
Generate a rectangular TLOF for a tiltrotor aircraft with 30m x 40m dimensions, elevation 5m, rotation 15 degrees, and 0.6 transparency. Location is [139.6917, 35.6895]. Add a 'V' landing marker in blue, scaled to 8, rotated to 90 degrees.

Example Output:
{{{{ 
  "TLOF": [
    {{{{
      "position": [
        139.6917,
        35.6895
      ],
      "dimensions": {{{{
        "unit": "m",
        "aircraft": "tiltrotor",
        "diameter": 1.0,
        "isVisible": true,
        "layerName": "Praveen_TLOF",
        "shapeType": "Rectangle",
        "scaleCategory": false,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": false,
        "sides": 3,
        "width": 30,
        "length": 30,
        "height": 0.2,
        "rotation": 15,
        "transparency": 0.6,
        "baseHeight": 5,
        "markingsCategory": true,
        "markingType": "solid",
        "markingColor": "white",
        "markingThickness": 0.5,
        "dashDistance": 1,
        "dashLength": 1,
        "landingMarkerCategory": true,
        "landingMarker": "V",
        "markerScale": 8,
        "markerThickness": 0.02,
        "markerRotation": 90,
        "markerColor": "blue",
        "letterThickness": 0.5,
        "tdpcCategory": false,
        "tdpcType": "circle",
        "tdpcScale": 5,
        "tdpcThickness": 0.5,
        "tdpcRotation": 0,
        "tdpcExtrusion": 0.02,
        "tdpcColor": "white",
        "lightCategory": true,
        "lightColor": "white",
        "lightScale": 1,
        "lightDistance": 1,
        "lightRadius": 0.3,
        "lightHeight": 0.2,
        "safetyAreaCategory": false,
        "safetyAreaType": "multiplier",
        "dValue": 10,
        "multiplier": 1.5,
        "offsetDistance": 3,
        "safetyNetCategory": false,
        "curveAngle": 45,
        "netHeight": 15,
        "safetyNetTransparency": 0.5
      }}}}
    }}}}
  ]
}}}}

Now generate the JSON for this input:"""),
    ("human", "{input}")
])

# Set up model and parser
parser = StrOutputParser()

# Build and invoke the chain
chain = prompt | model | parser

# Input to test
'''
response = chain.invoke({
    "input": "Generate a rectangular TLOF for a tiltrotor aircraft with 30m x 40m dimensions, elevation 5m, rotation 15 degrees, and 0.6 transparency. Location is [139.6917, 35.6895]. Add a 'V' landing marker in blue, scaled to 8, rotated to 90 degrees."
})
'''
# Ask user for the input
user_input = input("Enter your TLOF configuration request in natural language:\n")
response = chain.invoke({"input": user_input})

print("\nGenerated JSON Output:\n")
print(response)


Enter your TLOF configuration request in natural language:
 Generate a rectangular TLOF for a tiltrotor aircraft with 30m x 40m dimensions, elevation 5m, rotation 15 degrees, and 0.6 transparency. Location is [139.6917, 35.6895].                  Add a 'V' landing marker in blue, scaled to 8, rotated to 90 degrees.



Generated JSON Output:

```json
{
  "TLOF": [
    {
      "position": [
        139.6917,
        35.6895
      ],
      "dimensions": {
        "unit": "m",
        "aircraft": "tiltrotor",
        "shapeType": "Rectangle",
        "width": 30,
        "length": 40,
        "height": 0.2,
        "rotation": 15,
        "transparency": 0.6,
        "baseHeight": 5,
        "markingsCategory": true,
        "markingType": "solid",
        "markingColor": "white",
        "markingThickness": 0.5,
        "landingMarkerCategory": true,
        "landingMarker": "V",
        "markerScale": 8,
        "markerThickness": 0.02,
        "markerRotation": 90,
        "markerColor": "blue",
        "lightCategory": false
      }
    }
  ]
}
```


### JSON with Memory

In [66]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# System behavior
system_msg = SystemMessage(content="""
You are a civil aviation layout assistant that helps engineers, architects, and designers generate 3D-ready TLOF configurations in JSON format from plain English. Use defaults when needed, validate input, and return JSON only.
""")

# Conversation history
messages = [system_msg]

print("🚁 Welcome! I'm your TLOF assistant. How can I help you? Type 'exit' to quit.\n")

while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    # Add user message
    messages.append(HumanMessage(content=user_input))

    # Get model response
    response = model.invoke(messages)

    # Add AI response to history
    messages.append(AIMessage(content=response.content))

    print("\nAssistant:\n", response.content, "\n")


🚁 Welcome! I'm your TLOF assistant. How can I help you? Type 'exit' to quit.



User:  Design a circular TLOF (diameter 35m) for an air ambulance helicopter at [2.3522, 48.8566]. Use white dashed markings (1.2 thickness). Add white lights every 5 meters with radius 0.3m and height 0.2m. Include a safety net with 60-degree curve and net height of 10m.



Assistant:
 ```json
{
  "TLOF": {
    "shape": "circular",
    "diameter": 35,
    "location": {
      "longitude": 2.3522,
      "latitude": 48.8566
    },
    "markings": {
      "type": "dashed",
      "color": "white",
      "thickness": 1.2
    },
    "lights": {
      "spacing": 5,
      "color": "white",
      "radius": 0.3,
      "height": 0.2
    },
    "safety_net": {
      "curve": 60,
      "height": 10
    }
  }
}
``` 



User:  update the diameter to 45



Assistant:
 ```json
{
  "TLOF": {
    "shape": "circular",
    "diameter": 45,
    "location": {
      "longitude": 2.3522,
      "latitude": 48.8566
    },
    "markings": {
      "type": "dashed",
      "color": "white",
      "thickness": 1.2
    },
    "lights": {
      "spacing": 5,
      "color": "white",
      "radius": 0.3,
      "height": 0.2
    },
    "safety_net": {
      "curve": 60,
      "height": 10
    }
  }
}
``` 



User:  exit


### Full JSON output (only) with Memory

In [73]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Initial system prompt
system_message = SystemMessage(
    content="""You are a civil aviation layout assistant that helps engineers, architects, and designers generate 3D-ready TLOF (Touchdown and Lift-Off Area) configurations from plain English descriptions and that generates TLOF configurations in JSON format. 
You understand and convert natural language into a structured JSON format for heliport design, including parameters like geometry, markings, lighting, safety areas, and landing markers. 
You respect strict validation rules for shape types, colors, marker types, dimensions, and category activations.
Use default values where input is missing or unclear, and validate all parameters with explanations.

Example Input:
Generate a rectangular TLOF for a tiltrotor aircraft with 30m x 40m dimensions, elevation 5m, rotation 15 degrees, and 0.6 transparency. Location is [139.6917, 35.6895]. Add a 'V' landing marker in blue, scaled to 8, rotated to 90 degrees.

Example Output:
{{{{ 
  "TLOF": [
    {{{{
      "position": [
        139.6917,
        35.6895
      ],
      "dimensions": {{{{
        "unit": "m",
        "aircraft": "tiltrotor",
        "diameter": 1.0,
        "isVisible": true,
        "layerName": "Praveen_TLOF",
        "shapeType": "Rectangle",
        "scaleCategory": false,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": false,
        "sides": 3,
        "width": 30,
        "length": 30,
        "height": 0.2,
        "rotation": 15,
        "transparency": 0.6,
        "baseHeight": 5,
        "markingsCategory": true,
        "markingType": "solid",
        "markingColor": "white",
        "markingThickness": 0.5,
        "dashDistance": 1,
        "dashLength": 1,
        "landingMarkerCategory": true,
        "landingMarker": "V",
        "markerScale": 8,
        "markerThickness": 0.02,
        "markerRotation": 90,
        "markerColor": "blue",
        "letterThickness": 0.5,
        "tdpcCategory": false,
        "tdpcType": "circle",
        "tdpcScale": 5,
        "tdpcThickness": 0.5,
        "tdpcRotation": 0,
        "tdpcExtrusion": 0.02,
        "tdpcColor": "white",
        "lightCategory": true,
        "lightColor": "white",
        "lightScale": 1,
        "lightDistance": 1,
        "lightRadius": 0.3,
        "lightHeight": 0.2,
        "safetyAreaCategory": false,
        "safetyAreaType": "multiplier",
        "dValue": 10,
        "multiplier": 1.5,
        "offsetDistance": 3,
        "safetyNetCategory": false,
        "curveAngle": 45,
        "netHeight": 15,
        "safetyNetTransparency": 0.5
      }}}}
    }}}}
  ]
}}}}

Now generate the JSON for this input:"""
)

# Initialize message memory with the system message
messages = [system_message]

# Output parser
parser = StrOutputParser()

# Interactive assistant
print("🚁 Welcome! I'm your TLOF assistant. How can I help you? Type 'exit' to quit.\n")

while True:
    user_input = input("User: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        break

    # Append user's message
    messages.append(HumanMessage(content=user_input))

    # Invoke the model with full history
    response = model.invoke(messages)

    # Add AI response to memory
    messages.append(AIMessage(content=response.content))

    # Output the assistant's response
    print("\nAssistant:\n", response.content, "\n")


🚁 Welcome! I'm your TLOF assistant. How can I help you? Type 'exit' to quit.



User:  Design a circular TLOF (diameter 35m) for an air ambulance helicopter at [2.3522, 48.8566]. Use white dashed markings (1.2 thickness). Add white lights every 5 meters with radius 0.3m and height 0.2m. Include a safety net with 60-degree curve and net height of 10m.



Assistant:
 {
  "TLOF": [
    {
      "position": [
        2.3522,
        48.8566
      ],
      "dimensions": {
        "unit": "m",
        "aircraft": "helicopter",
        "diameter": 35,
        "isVisible": true,
        "layerName": "Praveen_TLOF",
        "shapeType": "Circle",
        "scaleCategory": false,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": false,
        "sides": 0,
        "width": 0,
        "length": 0,
        "height": 0.2,
        "rotation": 0,
        "transparency": 0,
        "baseHeight": 0,
        "markingsCategory": true,
        "markingType": "dashed",
        "markingColor": "white",
        "markingThickness": 1.2,
        "dashDistance": 5,
        "dashLength": 2,
        "landingMarkerCategory": false,
        "landingMarker": "",
        "markerScale": 0,
        "markerThickness": 0,
        "markerRotation": 0,
        "markerColor": "",
  

User:  update net height to 11



Assistant:
 {
  "TLOF": [
    {
      "position": [
        2.3522,
        48.8566
      ],
      "dimensions": {
        "unit": "m",
        "aircraft": "helicopter",
        "diameter": 35,
        "isVisible": true,
        "layerName": "Praveen_TLOF",
        "shapeType": "Circle",
        "scaleCategory": false,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": false,
        "sides": 0,
        "width": 0,
        "length": 0,
        "height": 0.2,
        "rotation": 0,
        "transparency": 0,
        "baseHeight": 0,
        "markingsCategory": true,
        "markingType": "dashed",
        "markingColor": "white",
        "markingThickness": 1.2,
        "dashDistance": 5,
        "dashLength": 2,
        "landingMarkerCategory": false,
        "landingMarker": "",
        "markerScale": 0,
        "markerThickness": 0,
        "markerRotation": 0,
        "markerColor": "",
  

User:  revert back 



Assistant:
 {
  "TLOF": [
    {
      "position": [
        2.3522,
        48.8566
      ],
      "dimensions": {
        "unit": "m",
        "aircraft": "helicopter",
        "diameter": 35,
        "isVisible": true,
        "layerName": "Praveen_TLOF",
        "shapeType": "Circle",
        "scaleCategory": false,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": false,
        "sides": 0,
        "width": 0,
        "length": 0,
        "height": 0.2,
        "rotation": 0,
        "transparency": 0,
        "baseHeight": 0,
        "markingsCategory": true,
        "markingType": "dashed",
        "markingColor": "white",
        "markingThickness": 1.2,
        "dashDistance": 5,
        "dashLength": 2,
        "landingMarkerCategory": false,
        "landingMarker": "",
        "markerScale": 0,
        "markerThickness": 0,
        "markerRotation": 0,
        "markerColor": "",
  

User:  exit


### Full and specific JSON output with Memory

In [ ]:
### Full and specific JSON output with Memory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Initial system prompt
system_message = SystemMessage(
    content="""You are a civil aviation layout assistant that helps engineers, architects, and designers generate 3D-ready TLOF (Touchdown and Lift-Off Area) configurations from plain English descriptions, producing TLOF configurations in JSON format.
You understand and convert natural language into a structured JSON format for heliport design, including parameters like geometry, markings, lighting, safety areas, and landing markers.
You respect strict validation rules for shape types, colors, marker types, dimensions, and category activations.
Use default values where input is missing or unclear, and validate all parameters with explanations.

**Instructions for Handling Updates**:
- When the user requests an update to an existing TLOF configuration (e.g., 'update net height to 11'), respond with a minimal JSON object that includes only the updated field(s) within the same nested structure as the original (e.g., `{"TLOF": [{"dimensions": {"netHeight": 11}}]}`).
- Do not regenerate or include unchanged fields unless explicitly requested.
- Ensure the JSON structure remains compatible with the full TLOF configuration, preserving the `TLOF` array and `dimensions` object hierarchy.
- For new TLOF configurations, provide the full JSON structure with all required fields, using defaults for unspecified parameters.

**Validation Rules**:
- Validate shape types (e.g., 'Rectangle', 'Circle'), colors (e.g., 'white', 'blue'), marker types (e.g., 'V', 'H'), and dimensions.
- Use default values for missing parameters (e.g., `transparency`: 0, `baseHeight`: 0, `markingColor`: 'white').
- Ensure numerical values (e.g., `diameter`, `netHeight`) are positive and within realistic bounds.

**Example Input for New Configuration**:
Generate a rectangular TLOF for a tiltrotor aircraft with 30m x 40m dimensions, elevation 5m, rotation 15 degrees, and 0.6 transparency. Location is [139.6917, 35.6895]. Add a 'V' landing marker in blue, scaled to 8, rotated to 90 degrees.

**Example Output for New Configuration**:
{
  "TLOF": [
    {
      "position": [
        139.6917,
        35.6895
      ],
      "dimensions": {
        "unit": "m",
        "aircraft": "tiltrotor",
        "diameter": 1.0,
        "isVisible": true,
        "layerName": "Praveen_TLOF",
        "shapeType": "Rectangle",
        "scaleCategory": false,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": false,
        "sides": 3,
        "width": 30,
        "length": 40,
        "height": 0.2,
        "rotation": 15,
        "transparency": 0.6,
        "baseHeight": 5,
        "markingsCategory": true,
        "markingType": "solid",
        "markingColor": "white",
        "markingThickness": 0.5,
        "dashDistance": 1,
        "dashLength": 1,
        "landingMarkerCategory": true,
        "landingMarker": "V",
        "markerScale": 8,
        "markerThickness": 0.02,
        "markerRotation": 90,
        "markerColor": "blue",
        "letterThickness": 0.5,
        "tdpcCategory": false,
        "tdpcType": "circle",
        "tdpcScale": 5,
        "tdpcThickness": 0.5,
        "tdpcRotation": 0,
        "tdpcExtrusion": 0.02,
        "tdpcColor": "white",
        "lightCategory": true,
        "lightColor": "white",
        "lightScale": 1,
        "lightDistance": 1,
        "lightRadius": 0.3,
        "lightHeight": 0.2,
        "safetyAreaCategory": false,
        "safetyAreaType": "multiplier",
        "dValue": 10,
        "multiplier": 1.5,
        "offsetDistance": 3,
        "safetyNetCategory": false,
        "curveAngle": 45,
        "netHeight": 15,
        "safetyNetTransparency": 0.5
      }
    }
  ]
}

**Example Input for Update**:
update net height to 11

**Example Output for Update**:
{
  "TLOF": [
    {
      "dimensions": {
        "netHeight": 11
      }
    }
  ]
}

Now generate the JSON for the user's input:"""
)

# Initialize message memory with the system message
messages = [system_message]

# Output parser
parser = StrOutputParser()

# Interactive assistant
print("🚁 Welcome! I'm your TLOF assistant. How can I help you? Type 'exit' to quit.\n")

while True:
    user_input = input("User: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        break

    # Append user's message
    messages.append(HumanMessage(content=user_input))

    # Invoke the model with full history
    response = model.invoke(messages)

    # Add AI response to memory
    messages.append(AIMessage(content=response.content))

    # Output the assistant's response
    print("\nAssistant:\n", response.content, "\n")


🚁 Welcome! I'm your TLOF assistant. How can I help you? Type 'exit' to quit.



User:  Build a rectangle TLOF for military aircraft (width 35m, length 45m) at [37.7749, -122.4194]. Use black solid markings, a black 'H' marker, and a fully opaque surface (transparency 1). No lighting or safety features are required.



Assistant:
 {
  "TLOF": [
    {
      "position": [
        37.7749,
        -122.4194
      ],
      "dimensions": {
        "unit": "m",
        "aircraft": "military",
        "diameter": 1.0,
        "isVisible": true,
        "layerName": "Praveen_TLOF",
        "shapeType": "Rectangle",
        "scaleCategory": false,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": false,
        "sides": 3,
        "width": 35,
        "length": 45,
        "height": 0.2,
        "rotation": 0,
        "transparency": 1,
        "baseHeight": 0,
        "markingsCategory": true,
        "markingType": "solid",
        "markingColor": "black",
        "markingThickness": 0.5,
        "dashDistance": 1,
        "dashLength": 1,
        "landingMarkerCategory": true,
        "landingMarker": "H",
        "markerScale": 1,
        "markerThickness": 0.02,
        "markerRotation": 0,
        "markerColor

User:  update the d value to 15



Assistant:
 {
  "TLOF": [
    {
      "dimensions": {
        "dValue": 15
      }
    }
  ]
} 



User:  revert back the d value



Assistant:
 {
  "TLOF": [
    {
      "dimensions": {
        "dValue": 10
      }
    }
  ]
} 



### Full and specific JSON output with Memory and ask questions back to user if prompt for vague

In [ ]:
### Full and specific JSON output with Memory and ask questions back to user if prompt for vague
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Initial system prompt
system_message = SystemMessage(
    content="""You are a civil aviation layout assistant that helps engineers, architects, and designers generate 3D-ready TLOF (Touchdown and Lift-Off Area) configurations from plain English descriptions, producing TLOF configurations in JSON format.
You understand and convert natural language into a structured JSON format for heliport design, including parameters like geometry, markings, lighting, safety areas, and landing markers.
You respect strict validation rules for shape types, colors, marker types, dimensions, and category activations.

**Instructions for Handling Incomplete Inputs**:
- If the user's input is vague or lacks essential parameters (e.g., shape, dimensions, location, or helicopter type), do not generate a JSON output. Instead, respond with a list of clarifying questions to gather the missing information.
- Essential parameters include:
  - Shape type (e.g., 'Rectangle', 'Circle').
  - Dimensions (e.g., width and length for Rectangle, diameter for Circle).
  - Location (geographic coordinates as [longitude, latitude]).
  - Helicopter type (e.g., 'light', 'medium', 'heavy', 'air ambulance').
- Example questions for incomplete input like 'create TLOF for a helicopter':
  - What is the shape of the TLOF (e.g., Rectangle or Circle)?
  - What are the dimensions of the TLOF (e.g., width and length for Rectangle, diameter for Circle)?
  - What is the location of the TLOF (please provide coordinates as [longitude, latitude])?
  - What type of helicopter is this TLOF for (e.g., light, medium, heavy, air ambulance)?
  - Do you want specific markings, lighting, or safety features (e.g., landing marker, lights, safety net)?
- Only generate a JSON output once all essential parameters are provided or clarified.

**Instructions for Handling Updates**:
- When the user requests an update to an existing TLOF configuration (e.g., 'update net height to 11'), respond with a minimal JSON object that includes only the updated field(s) within the same nested structure as the original (e.g., `{"TLOF": [{"dimensions": {"netHeight": 11}}]}`).
- Do not regenerate or include unchanged fields unless explicitly requested.
- Ensure the JSON structure remains compatible with the full TLOF configuration, preserving the `TLOF` array and `dimensions` object hierarchy.

**Instructions for New Configurations**:
- For new TLOF configurations with complete inputs, provide the full JSON structure with all required fields, using defaults for unspecified parameters.
- Use default values for missing parameters (e.g., `transparency`: 0, `baseHeight`: 0, `markingColor`: 'white').

**Validation Rules**:
- Validate shape types (e.g., 'Rectangle', 'Circle'), colors (e.g., 'white', 'blue'), marker types (e.g., 'V', 'H'), and dimensions.
- Ensure numerical values (e.g., `diameter`, `netHeight`) are positive and within realistic bounds.
- For helicopters, ensure dimensions are appropriate for the specified type (e.g., light: ~15-20m diameter, medium: ~25-30m, heavy: ~35-40m).

**Example Input for Incomplete Request**:
create TLOF for a helicopter

**Example Output for Incomplete Request**:
Please provide more details to generate the TLOF configuration:
- What is the shape of the TLOF (e.g., Rectangle or Circle)?
- What are the dimensions of the TLOF (e.g., width and length for Rectangle, diameter for Circle)?
- What is the location of the TLOF (please provide coordinates as [longitude, latitude])?
- What type of helicopter is this TLOF for (e.g., light, medium, heavy, air ambulance)?
- Do you want specific markings, lighting, or safety features (e.g., landing marker, lights, safety net)?

**Example Input for New Configuration**:
Generate a rectangular TLOF for a tiltrotor aircraft with 30m x 40m dimensions, elevation 5m, rotation 15 degrees, and 0.6 transparency. Location is [139.6917, 35.6895]. Add a 'V' landing marker in blue, scaled to 8, rotated to 90 degrees.

**Example Output for New Configuration**:
{
  "TLOF": [
    {
      "position": [
        139.6917,
        35.6895
      ],
      "dimensions": {
        "unit": "m",
        "aircraft": "tiltrotor",
        "diameter": 1.0,
        "isVisible": true,
        "layerName": "Praveen_TLOF",
        "shapeType": "Rectangle",
        "scaleCategory": false,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": false,
        "sides": 3,
        "width": 30,
        "length": 40,
        "height": 0.2,
        "rotation": 15,
        "transparency": 0.6,
        "baseHeight": 5,
        "markingsCategory": true,
        "markingType": "solid",
        "markingColor": "white",
        "markingThickness": 0.5,
        "dashDistance": 1,
        "dashLength": 1,
        "landingMarkerCategory": true,
        "landingMarker": "V",
        "markerScale": 8,
        "markerThickness": 0.02,
        "markerRotation": 90,
        "markerColor": "blue",
        "letterThickness": 0.5,
        "tdpcCategory": false,
        "tdpcType": "circle",
        "tdpcScale": 5,
        "tdpcThickness": 0.5,
        "tdpcRotation": 0,
        "tdpcExtrusion": 0.02,
        "tdpcColor": "white",
        "lightCategory": true,
        "lightColor": "white",
        "lightScale": 1,
        "lightDistance": 1,
        "lightRadius": 0.3,
        "lightHeight": 0.2,
        "safetyAreaCategory": false,
        "safetyAreaType": "multiplier",
        "dValue": 10,
        "multiplier": 1.5,
        "offsetDistance": 3,
        "safetyNetCategory": false,
        "curveAngle": 45,
        "netHeight": 15,
        "safetyNetTransparency": 0.5
      }
    }
  ]
}

**Example Input for Update**:
update net height to 11

**Example Output for Update**:
{
  "TLOF": [
    {
      "dimensions": {
        "netHeight": 11
      }
    }
  ]
}

Now process the user's input:"""
)

# Initialize message memory with the system message
messages = [system_message]

# Output parser
parser = StrOutputParser()

# Interactive assistant
print("🚁 Welcome! I'm your TLOF assistant. How can I help you? Type 'exit' to quit.\n")

while True:
    user_input = input("User: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        break

    # Append user's message
    messages.append(HumanMessage(content=user_input))

    # Invoke the model with full history
    response = model.invoke(messages)

    # Add AI response to memory
    messages.append(AIMessage(content=response.content))

    # Output the assistant's response
    print("\nAssistant:\n", response.content, "\n")


🚁 Welcome! I'm your TLOF assistant. How can I help you? Type 'exit' to quit.



### RunnableWithMessageHistory

In [75]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
from langchain_core.runnables import Runnable
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory


# 1️⃣ Define system prompt and template
system_message = SystemMessage(
    content="""You are a civil aviation layout assistant that helps engineers, architects, and designers generate 3D-ready TLOF configurations in JSON format.
Use structured output with validated fields for aircraft geometry, markings, lighting, safety nets, and landing markers. Use sensible defaults and explain any assumptions."""
)

prompt = ChatPromptTemplate.from_messages([
    system_message,
    ("human", "{input}")
])

# 2️⃣ Define model and output parser
# model = ChatOpenAI(model="gpt-4")
parser = StrOutputParser()

# 3️⃣ Create base runnable chain
chain: Runnable = prompt | model | parser

# 4️⃣ Define a simple memory backend
def get_memory(session_id: str) -> BaseChatMessageHistory:
    return InMemoryChatMessageHistory()  # Can be swapped for Redis, Postgres, etc.

# 5️⃣ Wrap the chain with memory support
chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_memory,
    input_messages_key="input",  # key in the input dict to track
    history_messages_key="history",  # key to inject past messages into prompt
)

# 6️⃣ Run interaction loop
print("🚁 TLOF Assistant with Memory. Type 'exit' to quit.\n")
session_id = "user-123"  # could be per-user or per-convo

while True:
    user_input = input("User: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        break

    response = chain_with_memory.invoke(
        {"input": user_input},
        config={"configurable": {"session_id": session_id}}  # session-aware memory
    )

    print("\nAssistant:\n", response, "\n")


🚁 TLOF Assistant with Memory. Type 'exit' to quit.



User:  Design a circular TLOF (diameter 35m) for an air ambulance helicopter at [2.3522, 48.8566]. Use white dashed markings (1.2 thickness). Add white lights every 5 meters with radius 0.3m and height 0.2m. Include a safety net with 60-degree curve and net height of 10m.



Assistant:
 Here is the JSON configuration for the specified TLOF:

```json
{
  "TLOF": {
    "type": "circular",
    "location": {
      "latitude": 48.8566,
      "longitude": 2.3522
    },
    "diameter": 35,
    "markings": {
      "color": "white",
      "pattern": "dashed",
      "thickness": 1.2
    },
    "lighting": {
      "spacing": 5,
      "lights": [
        {"position": 0, "color": "white", "radius": 0.3, "height": 0.2},
        {"position": 5, "color": "white", "radius": 0.3, "height": 0.2},
        {"position": 10, "color": "white", "radius": 0.3, "height": 0.2},
        {"position": 15, "color": "white", "radius": 0.3, "height": 0.2},
        {"position": 20, "color": "white", "radius": 0.3, "height": 0.2},
        {"position": 25, "color": "white", "radius": 0.3, "height": 0.2},
        {"position": 30, "color": "white", "radius": 0.3, "height": 0.2}
      ]
    },
    "safety_net": {
      "curve": 60,
      "height": 10
    }
  }
}
```

### Explanation of Assumpti

User:  exit
